### Using callbacks to act on a model during training

A callback is an object (a class instance implementing specific methods) that is passed to the model in the call to fit and that is called by the model at various points during training. It has access to all the available data about the state of the model and its per- formance, and it can take action: interrupt training, save a model, load a different weight set, or otherwise alter the state of the model.
Callbacks can be used for:
* Model checkpointing: save current weights
* Early stopping: interrupting when validation loss is no longer improving
* Dynamically adjust parameters, such as the learning rate
* Logging metrics during training

Let's now go through an example where we use the `Model Checkpoint` and `Early stopping` callbacks

In [ ]:
# (not run)
import keras

# here we define the list of callbacks that need to be used during fit.
# the first one would stop the training if the accuracy stops increasing
# the second would monitor validation loss and save a new model every time this improves
# (but would not overwrite the model if it does not)
callback_list = [keras.callbacks.EarlyStopping(monitor='acc',patience=1),
                 keras.callbacks.ModelCheckpoint(filepath='/home/ec2-user/models/callbacks/example1.h5',
                                                 monitor='val_loss', save_best_only=True,)]

model.compile(optmizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(x,y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

The following is an example for the `ReduceLROnPlateau` callback

In [ ]:
# (not run)
# in this case the callback would reduce the learning rate (to 10% of its original value) 
# if it notices a plateau in validation loss for 10 epochs
callbacks_list = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)]

model.fit(x,y,epochs=100, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

It is also possible to write your own callbacks. Callbacks are implemented by sub-classing the class `keras.callbacks.Callback`. (See book on page 251 for details).
The following is an example of a custom callback that saves to disk (as Numpy arrays) the activations of every layer of the model at the end of every epoch, computed on the first sample of the validation set.

In [3]:
# (not run)
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    #Called by the parent model before training, to inform the callback of what model will be calling it
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()                           

### Introduction to Tensorboard

The key purpose of TensorBoard is to help you visually monitor everything that goes on inside your model during training. If you’re monitoring more information than just the model’s final loss, you can develop a clearer vision of what the model does and doesn’t do, and you can make progress more quickly. TensorBoard gives you access to several neat features, all in your browser:
* Visually monitoring metrics during training
* Visualizing your model architecture
* Visualizing histograms of activations and gradients
* Exploring embeddings in 3D

We demonstrate this by training a 1D convnet on the IMDB sentiment-analysis task.

In [4]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

# Number of words to consider as features
max_features = 2000
# Maximum length of text to consider in each review (number of words)
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [12]:
# To use TensorBoard we need to create a directory where to save the output
# And we need to use the appropriate callback.
# This callback will record activation histograms and embeddings once every epoch
callbacks = [keras.callbacks.TensorBoard(log_dir="/home/ec2-user/tensorboard",
                                         histogram_freq=1,
                                         embeddings_freq=1)]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0894 - acc: 0.0328 - val_loss: 1.5641 - val_acc: 0.1368
Epoch 2/20
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0942 - acc: 0.0340 - val_loss: 1.5579 - val_acc: 0.1276
Epoch 3/20
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0898 - acc: 0.0314 - val_loss: 1.5985 - val_acc: 0.1254
Epoch 4/20
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0908 - acc: 0.0305 - val_loss: 1.6225 - val_acc: 0.1206
Epoch 5/20
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0898 - acc: 0.0251 - val_loss: 1.6049 - val_acc: 0.1170
Epoch 6/20
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0950 - acc: 0.0271 - val_loss: 1.6112 - val_acc: 0.1186
Epoch 7/20
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0866 - acc: 0.0209 - val_loss: 1.5729 - val_acc: